In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import optuna

In [ ]:
import warnings
warnings.filterwarnings( 'ignore' )

# KS test 변수선택

In [ ]:
sk=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/SK_label.csv')
hd=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/hd_label.csv')
sh=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/shinhan_label.csv')

In [ ]:
le = LabelEncoder()

le.fit(sk['day1_label'].values)
sk['day1_label'] = le.transform(sk['day1_label'].values)
hd['day1_label'] = le.transform(hd['day1_label'].values)
sh['day1_label'] = le.transform(sh['day1_label'].values)

In [ ]:
sk.columns

Index(['날짜', 'year', 'month', 'day', '종가', '대비', '등락률', '거래량', '거래대금', '시가총액',
       '외국인 보유수량', '외국인 지분율', '토론방', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인',
       '검색어', '보도량', '기사감성점수', '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동',
       '코스피종가', '코스피대비', '코스피등락률', '코스피거래량', '코스피거래대금', '코스피시가총액', '한은금리',
       '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로', '경제심리지수(원계열)', '경제심리지수(순환변동치)',
       '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수', '경제활동참가율(%)', '실업률(%)',
       '고용률(%)', 'day1_label', '3일 등락률', 'day3_label'],
      dtype='object')

In [ ]:
hd.columns

Index(['날짜', 'year', 'month', 'day', '종가', '대비', '등락률', '거래량', '거래대금', '시가총액',
       '외국인 보유수량', '외국인 지분율', '토론방', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인',
       '검색어', '보도량', '기사감성점수', '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동',
       '코스피종가', '코스피대비', '코스피등락률', '코스피거래량', '코스피거래대금', '코스피시가총액', '한은금리',
       '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로', '경제심리지수(원계열)', '경제심리지수(순환변동치)',
       '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수', '경제활동참가율(%)', '실업률(%)',
       '고용률(%)', 'day1_label', '3일_등락률', 'day3_label'],
      dtype='object')

In [ ]:
sh = sh.drop('year-month', axis=1)
sh.columns

Index(['날짜', 'year', 'month', 'day', '종가', '대비', '등락률', '거래량', '거래대금', '시가총액',
       '외국인 보유수량', '외국인 지분율', '토론방', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인',
       '검색어', '보도량', '기사감성점수', '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동',
       '코스피종가', '코스피대비', '코스피등락률', '코스피거래량', '코스피거래대금', '코스피시가총액', '한은금리',
       '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로', '경제심리지수(원계열)', '경제심리지수(순환변동치)',
       '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수', '경제활동참가율(%)', '실업률(%)',
       '고용률(%)', 'day1_label', '3일_등락률', 'day3_label'],
      dtype='object')

In [ ]:
sk = sk.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일 등락률'], axis=1)
hd = hd.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일_등락률'], axis=1)
sh = sh.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일_등락률'], axis=1)

In [ ]:
sk0 = sk[sk['day1_label']==1]
sk1 = sk[sk['day1_label'].isin([0,2])]

hd0 = hd[hd['day1_label']==1]
hd1 = hd[hd['day1_label'].isin([0,2])] 

sh0 = sh[sh['day1_label']==1]
sh1 = sh[sh['day1_label'].isin([0,2])]

In [ ]:
from scipy.stats import kstest

maint = []
cols = sk.drop(['day1_label', 'day3_label'], axis=1).columns

for c in cols:
    rst_sk = kstest(sk0[c], sk1[c])
    rst_hd = kstest(hd0[c], hd1[c])
    rst_sh = kstest(sh0[c], sh1[c])

    if (rst_sk[1]<0.05) and (rst_hd[1]<0.05) and (rst_sh[1]<0.05):
        maint.append(c)

maint #유지와 매도/매수 분포 다른 변수

['등락률',
 '거래량',
 '거래대금',
 '토론방',
 '순매수_기관',
 '순매수_기타법인',
 '순매수_개인',
 '순매수_외국인',
 '검색어',
 '보도량',
 '코스피등락률',
 '경제활동참가율(%)',
 '실업률(%)',
 '고용률(%)']

In [ ]:
sk0 = sk[sk['day1_label']==0]
sk1 = sk[sk['day1_label']==2]

hd0 = hd[hd['day1_label']==0]
hd1 = hd[hd['day1_label']==2] 

sh0 = sh[sh['day1_label']==0]
sh1 = sh[sh['day1_label']==2]

In [ ]:
from scipy.stats import kstest

buysell = []
cols = sk.drop(['day1_label', 'day3_label'], axis=1).columns

for c in cols:
    rst_sk = kstest(sk0[c], sk1[c])
    rst_hd = kstest(hd0[c], hd1[c])
    rst_sh = kstest(sh0[c], sh1[c])

    if (rst_sk[1]<0.05) and (rst_hd[1]<0.05) and (rst_sh[1]<0.05):
        buysell.append(c)

buysell #매도와 매수 분포 다른 변수

['등락률', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인', '코스피등락률']

# XGB 이중분류 FM

In [ ]:
sk=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/SK_label.csv')
hd=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/hd_label.csv')
sh=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/shinhan_label.csv')

In [ ]:
sk = sk.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일 등락률'], axis=1)
hd = hd.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일_등락률'], axis=1)
sh = sh.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일_등락률', 'year-month'], axis=1)

In [ ]:
sk = sk.drop('day3_label', axis=1)
hd = hd.drop('day3_label', axis=1)
sh = sh.drop('day3_label', axis=1)

len(sk), len(hd), len(sh)

(1429, 1423, 1429)

In [ ]:
sk.columns

Index(['종가', '등락률', '거래량', '거래대금', '시가총액', '외국인 보유수량', '외국인 지분율', '토론방',
       '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인', '검색어', '보도량', '기사감성점수',
       '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동', '코스피종가', '코스피등락률', '코스피거래량',
       '코스피거래대금', '코스피시가총액', '한은금리', '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로',
       '경제심리지수(원계열)', '경제심리지수(순환변동치)', '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수',
       '경제활동참가율(%)', '실업률(%)', '고용률(%)', 'day1_label'],
      dtype='object')

In [ ]:
hd.columns

Index(['종가', '등락률', '거래량', '거래대금', '시가총액', '외국인 보유수량', '외국인 지분율', '토론방',
       '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인', '검색어', '보도량', '기사감성점수',
       '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동', '코스피종가', '코스피등락률', '코스피거래량',
       '코스피거래대금', '코스피시가총액', '한은금리', '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로',
       '경제심리지수(원계열)', '경제심리지수(순환변동치)', '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수',
       '경제활동참가율(%)', '실업률(%)', '고용률(%)', 'day1_label'],
      dtype='object')

In [ ]:
sh.columns

Index(['종가', '등락률', '거래량', '거래대금', '시가총액', '외국인 보유수량', '외국인 지분율', '토론방',
       '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인', '검색어', '보도량', '기사감성점수',
       '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동', '코스피종가', '코스피등락률', '코스피거래량',
       '코스피거래대금', '코스피시가총액', '한은금리', '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로',
       '경제심리지수(원계열)', '경제심리지수(순환변동치)', '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수',
       '경제활동참가율(%)', '실업률(%)', '고용률(%)', 'day1_label'],
      dtype='object')

In [ ]:
label=pd.DataFrame()
label['encoding'] = sk['등락률'].apply(lambda x: 0 if abs(x)<3 else 1)
#유지 vs 매도/매수로 이진분류
#유지=0
#매도/매수=1

sk1 = sk.drop('day1_label', axis=1)

label['encoding'].value_counts()

0    1177
1     252
Name: encoding, dtype: int64

In [ ]:
sk1 = sk1.shift(1) # 한 행씩 데이터 아래로 내리기
df=pd.concat([sk1,label], axis=1)
df=df.drop(0,axis=0)
df=df.reset_index(drop=True)
df

,종가,등락률,거래량,거래대금,시가총액,외국인 보유수량,외국인 지분율,토론방,순매수_기관,순매수_기타법인,...,경제심리지수(원계열),경제심리지수(순환변동치),산업생산지수,물가상승률,소비자신뢰지수,소비자심리지수,경제활동참가율(%),실업률(%),고용률(%),encoding
0,58800.0,2.26,3431811.0,1.987173e+11,4.280654e+13,373079472.0,51.25,233.0,3.938152e+09,1.145514e+09,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,0
1,59300.0,0.85,4441904.0,2.638198e+11,4.317054e+13,373205666.0,51.26,104.0,3.378250e+10,-1.557680e+08,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,0
2,60600.0,2.19,4549233.0,2.736022e+11,4.411694e+13,372839313.0,51.21,222.0,4.528110e+10,-2.953806e+09,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,0
3,60500.0,-0.17,3932203.0,2.386388e+11,4.404414e+13,372949640.0,51.23,131.0,3.251843e+10,1.726089e+09,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,1
4,62800.0,3.80,5315543.0,3.319266e+11,4.571855e+13,372011865.0,51.10,168.0,4.404935e+10,-3.212170e+07,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,87300.0,-1.36,4670908.0,4.121193e+11,6.355461e+13,366220750.0,50.30,155.0,2.809119e+10,1.655750e+09,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,0
1424,85500.0,-2.06,3211190.0,2.751969e+11,6.224420e+13,365466692.0,50.20,84.0,-2.786144e+09,-1.850354e+09,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,1
1425,88400.0,3.39,3180431.0,2.779770e+11,6.435541e+13,365123199.0,50.15,69.0,5.638837e+10,4.898941e+08,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,0
1426,86900.0,-1.70,3070422.0,2.675525e+11,6.326341e+13,365150714.0,50.16,90.0,2.506547e+10,-2.235300e+08,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,0


In [ ]:
x = df.drop(['encoding'], axis=1)
y = df['encoding']

In [ ]:
x

,종가,등락률,거래량,거래대금,시가총액,외국인 보유수량,외국인 지분율,토론방,순매수_기관,순매수_기타법인,...,원/유로,경제심리지수(원계열),경제심리지수(순환변동치),산업생산지수,물가상승률,소비자신뢰지수,소비자심리지수,경제활동참가율(%),실업률(%),고용률(%)
0,58800.0,2.26,3431811.0,1.987173e+11,4.280654e+13,373079472.0,51.25,233.0,3.938152e+09,1.145514e+09,...,1261.29,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5
1,59300.0,0.85,4441904.0,2.638198e+11,4.317054e+13,373205666.0,51.26,104.0,3.378250e+10,-1.557680e+08,...,1265.33,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5
2,60600.0,2.19,4549233.0,2.736022e+11,4.411694e+13,372839313.0,51.21,222.0,4.528110e+10,-2.953806e+09,...,1262.25,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5
3,60500.0,-0.17,3932203.0,2.386388e+11,4.404414e+13,372949640.0,51.23,131.0,3.251843e+10,1.726089e+09,...,1251.75,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5
4,62800.0,3.80,5315543.0,3.319266e+11,4.571855e+13,372011865.0,51.10,168.0,4.404935e+10,-3.212170e+07,...,1269.24,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,87300.0,-1.36,4670908.0,4.121193e+11,6.355461e+13,366220750.0,50.30,155.0,2.809119e+10,1.655750e+09,...,1394.42,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2
1424,85500.0,-2.06,3211190.0,2.751969e+11,6.224420e+13,365466692.0,50.20,84.0,-2.786144e+09,-1.850354e+09,...,1389.72,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2
1425,88400.0,3.39,3180431.0,2.779770e+11,6.435541e+13,365123199.0,50.15,69.0,5.638837e+10,4.898941e+08,...,1404.58,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2
1426,86900.0,-1.70,3070422.0,2.675525e+11,6.326341e+13,365150714.0,50.16,90.0,2.506547e+10,-2.235300e+08,...,1406.51,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2


In [ ]:
y

0       0
1       0
2       0
3       1
4       0
       ..
1423    0
1424    1
1425    0
1426    0
1427    0
Name: encoding, Length: 1428, dtype: int64

In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=5)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]

    #print(len(y_train), len(y_test))
    
    xgb_model = xgb.XGBClassifier(n_estimators=500, max_depth=4, random_state = 42).fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)

    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.7310924369747899
Fold 1 - f1_score: 0.6948426429395289

Fold 2 - Accuracy: 0.7436974789915967
Fold 2 - f1_score: 0.7154919384021552

Fold 3 - Accuracy: 0.6932773109243697
Fold 3 - f1_score: 0.6830748486279455

Fold 4 - Accuracy: 0.8067226890756303
Fold 4 - f1_score: 0.7781224639388465

Fold 5 - Accuracy: 0.7100840336134454
Fold 5 - f1_score: 0.7269000677125741



In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=5)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]
    
    xgb_model = xgb.XGBClassifier(n_estimators=500, max_depth=4,
                                  scale_pos_weight = 4, random_state = 42)
    xgb_model.fit(x_train, y_train)

    y_pred = xgb_model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.7478991596638656
Fold 1 - f1_score: 0.7046458146365342

Fold 2 - Accuracy: 0.7647058823529411
Fold 2 - f1_score: 0.7257606490872212

Fold 3 - Accuracy: 0.6848739495798319
Fold 3 - f1_score: 0.6954364094323526

Fold 4 - Accuracy: 0.8067226890756303
Fold 4 - f1_score: 0.7828397838008443

Fold 5 - Accuracy: 0.7184873949579832
Fold 5 - f1_score: 0.7281571043794919



In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

def objective(trial):
    param={
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 6)
    }
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train = x.iloc[train_index],y.iloc[train_index]
        x_test, y_test = x.iloc[test_index],y.iloc[test_index]
        
        xgb_model = xgb.XGBClassifier(**param, random_state = 42)
        xgb_model.fit(x_train, y_train)
        y_pred = xgb_model.predict(x_test)

    return (f1_score(y_test, y_pred, average='weighted'))

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-05-06 08:08:25,760] A new study created in memory with name: no-name-5a9d4462-5dba-4a41-97f2-379e4199dcb6
[I 2023-05-06 08:08:29,147] Trial 0 finished with value: 0.7312632577302766 and parameters: {'max_depth': 15, 'learning_rate': 0.6308348508587921, 'n_estimators': 813, 'min_child_weight': 7, 'gamma': 1.5311324404167802e-06, 'subsample': 0.351401263040105, 'colsample_bytree': 0.16775970782636992, 'reg_alpha': 3.834706061123105e-08, 'reg_lambda': 0.0003084707626560973, 'scale_pos_weight': 3}. Best is trial 0 with value: 0.7312632577302766.
[I 2023-05-06 08:08:32,473] Trial 1 finished with value: 0.773358090388775 and parameters: {'max_depth': 1, 'learning_rate': 0.0034496789761361535, 'n_estimators': 988, 'min_child_weight': 6, 'gamma': 2.1290566806612365e-05, 'subsample': 0.04633687381099051, 'colsample_bytree': 0.05255803977051032, 'reg_alpha': 0.0013719870730528778, 'reg_lambda': 5.596721769541997e-05, 'scale_pos_weight': 3}. Best is trial 1 with value: 0.773358090388775.


In [ ]:
print(study.best_params)
print(study.best_value)

{'max_depth': 9, 'learning_rate': 0.01100385355746383, 'n_estimators': 793, 'min_child_weight': 3, 'gamma': 0.08111344939307713, 'subsample': 0.17396801090934844, 'colsample_bytree': 0.35817658966946386, 'reg_alpha': 0.6464969164029837, 'reg_lambda': 0.11964080388328105, 'scale_pos_weight': 5}
0.7968945449576726


In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=5)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]
    
    xgb_model=xgb.XGBClassifier(**study.best_params, random_state = 42)
    xgb_model.fit(x_train, y_train)

    y_pred = xgb_model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.6764705882352942
Fold 1 - f1_score: 0.6873147136838821

Fold 2 - Accuracy: 0.7815126050420168
Fold 2 - f1_score: 0.7370723012042952

Fold 3 - Accuracy: 0.5672268907563025
Fold 3 - f1_score: 0.6062381191792956

Fold 4 - Accuracy: 0.8529411764705882
Fold 4 - f1_score: 0.7968520741629985

Fold 5 - Accuracy: 0.8067226890756303
Fold 5 - f1_score: 0.7968945449576726



In [ ]:
confusion_matrix(y_test, y_pred)

array([[182,  19],
       [ 27,  10]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('유지 정확도 : ', cm[0,0]/sum(cm[0]))
print('매수/매도 정확도 : ',cm[1,1]/sum(cm[1]))

유지 정확도 :  0.9054726368159204
매수/매도 정확도 :  0.2702702702702703


# XGB 다중분류 FM

In [ ]:
sk=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/SK_label.csv')
hd=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/hd_label.csv')
sh=pd.read_csv('/content/drive/MyDrive/2023-1학기-시계열팀주제분석/PreprocessedData/labeled/shinhan_label.csv')

In [ ]:
sk = sk.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일 등락률'], axis=1)
hd = hd.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일_등락률'], axis=1)
sh = sh.drop(['날짜', 'year', 'month', 'day', '대비', '코스피대비', '3일_등락률', 'year-month'], axis=1)

In [ ]:
sk = sk.drop('day3_label', axis=1)
hd = hd.drop('day3_label', axis=1)
sh = sh.drop('day3_label', axis=1)

len(sk), len(hd), len(sh)

(1429, 1423, 1429)

In [ ]:
le = LabelEncoder()

le.fit(sk['day1_label'].values)
sk['day1_label'] = le.transform(sk['day1_label'].values)

In [ ]:
label=pd.DataFrame()
label['encoding'] = sk['day1_label']
sk1 = sk.drop('day1_label', axis=1)

In [ ]:
label['encoding'].value_counts()

1    1177
0     135
2     117
Name: encoding, dtype: int64

In [ ]:
sk1 = sk1.shift(1) # 한 행씩 데이터 아래로 내리기
df=pd.concat([sk1,label], axis=1)
df=df.drop(0,axis=0)
df=df.reset_index(drop=True)
df

,종가,등락률,거래량,거래대금,시가총액,외국인 보유수량,외국인 지분율,토론방,순매수_기관,순매수_기타법인,...,경제심리지수(원계열),경제심리지수(순환변동치),산업생산지수,물가상승률,소비자신뢰지수,소비자심리지수,경제활동참가율(%),실업률(%),고용률(%),encoding
0,58800.0,2.26,3431811.0,1.987173e+11,4.280654e+13,373079472.0,51.25,233.0,3.938152e+09,1.145514e+09,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,1
1,59300.0,0.85,4441904.0,2.638198e+11,4.317054e+13,373205666.0,51.26,104.0,3.378250e+10,-1.557680e+08,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,1
2,60600.0,2.19,4549233.0,2.736022e+11,4.411694e+13,372839313.0,51.21,222.0,4.528110e+10,-2.953806e+09,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,1
3,60500.0,-0.17,3932203.0,2.386388e+11,4.404414e+13,372949640.0,51.23,131.0,3.251843e+10,1.726089e+09,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,0
4,62800.0,3.80,5315543.0,3.319266e+11,4.571855e+13,372011865.0,51.10,168.0,4.404935e+10,-3.212170e+07,...,99.8,99.6,103.9,97.338,111.0,110.8,63.9,3.8,61.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,87300.0,-1.36,4670908.0,4.121193e+11,6.355461e+13,366220750.0,50.30,155.0,2.809119e+10,1.655750e+09,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,1
1424,85500.0,-2.06,3211190.0,2.751969e+11,6.224420e+13,365466692.0,50.20,84.0,-2.786144e+09,-1.850354e+09,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,0
1425,88400.0,3.39,3180431.0,2.779770e+11,6.435541e+13,365123199.0,50.15,69.0,5.638837e+10,4.898941e+08,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,1
1426,86900.0,-1.70,3070422.0,2.675525e+11,6.326341e+13,365150714.0,50.16,90.0,2.506547e+10,-2.235300e+08,...,91.5,89.8,112.5,110.560,92.0,92.0,64.1,2.9,62.2,1


In [ ]:
x = df.drop(['encoding'], axis=1)
y = df['encoding']

In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train=x.iloc[train_index],y.iloc[train_index]
    x_test, y_test=x.iloc[test_index],y.iloc[test_index]
    
    params = {'objective': 'multi:softmax', 'num_class': 3}
    xgb_model = xgb.XGBClassifier(**params).fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)

    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred,average='weighted')}")
    print()

Fold 1 - Accuracy: 0.7815126050420168
Fold 1 - f1_score: 0.7168823927675336

Fold 2 - Accuracy: 0.7815126050420168
Fold 2 - f1_score: 0.7000278784499582

Fold 3 - Accuracy: 0.7058823529411765
Fold 3 - f1_score: 0.6862166649561982

Fold 4 - Accuracy: 0.8571428571428571
Fold 4 - f1_score: 0.7989657401422108

Fold 5 - Accuracy: 0.7941176470588235
Fold 5 - f1_score: 0.7574757839603115



In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

def objective(trial):
    param={
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 1e-8, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0)
    }
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train = x.iloc[train_index],y.iloc[train_index]
        x_test, y_test = x.iloc[test_index],y.iloc[test_index]
        
        xgb_model = xgb.XGBClassifier(**param, random_state = 42)
        xgb_model.fit(x_train, y_train)
        y_pred = xgb_model.predict(x_test)

    return (f1_score(y_test, y_pred, average='weighted'))

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-05-06 08:38:17,540] A new study created in memory with name: no-name-d155a072-be68-4191-bd3e-fac940e83fe7
[I 2023-05-06 08:38:21,271] Trial 0 finished with value: 0.015643180349062702 and parameters: {'max_depth': 15, 'learning_rate': 0.013226317712113415, 'n_estimators': 27, 'min_child_weight': 1, 'gamma': 1.371261723403911e-07, 'subsample': 8.302980911333643e-08, 'colsample_bytree': 2.5452503447529957e-08, 'reg_alpha': 1.7568362470144723e-08, 'reg_lambda': 2.7472264093270965e-07}. Best is trial 0 with value: 0.015643180349062702.
[I 2023-05-06 08:38:24,495] Trial 1 finished with value: 0.015643180349062702 and parameters: {'max_depth': 12, 'learning_rate': 0.009439265898159709, 'n_estimators': 656, 'min_child_weight': 4, 'gamma': 1.0419429978241805e-08, 'subsample': 3.418357589816546e-05, 'colsample_bytree': 3.54286846349917e-08, 'reg_alpha': 0.00045779361005107824, 'reg_lambda': 2.085792358542044e-06}. Best is trial 0 with value: 0.015643180349062702.
[I 2023-05-06 08:38:28,

In [ ]:
print(study.best_params)
print(study.best_value)

{'max_depth': 19, 'learning_rate': 0.06512230814631131, 'n_estimators': 295, 'min_child_weight': 1, 'gamma': 0.0005508384133406559, 'subsample': 0.05804924660842636, 'colsample_bytree': 0.00020022503635305395, 'reg_alpha': 2.5721341365372974e-05, 'reg_lambda': 0.000102680808628449}
0.7879258431911049


In [ ]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index]
    x_test, y_test = x.iloc[test_index], y.iloc[test_index]
    
    xgb_model=xgb.XGBClassifier(**study.best_params, random_state = 42)
    xgb_model.fit(x_train, y_train)

    y_pred = xgb_model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.8067226890756303
Fold 1 - f1_score: 0.7241743208911471

Fold 2 - Accuracy: 0.7899159663865546
Fold 2 - f1_score: 0.7009113504556753

Fold 3 - Accuracy: 0.7058823529411765
Fold 3 - f1_score: 0.6870985065757049

Fold 4 - Accuracy: 0.7899159663865546
Fold 4 - f1_score: 0.7657538309441424

Fold 5 - Accuracy: 0.8361344537815126
Fold 5 - f1_score: 0.7879258431911049



In [ ]:
confusion_matrix(y_test, y_pred)

array([[  3,  19,   0],
       [  3, 196,   2],
       [  0,  15,   0]])

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ',cm[2,2]/sum(cm[2]))

매수 정확도 :  0.13636363636363635
매도 정확도 :  0.0
